# Forecasting de Demanda

## Librerías

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

## Generar series temporales sintéticas

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

# Cargar métricas base
df = pd.read_csv("metricas_venta_integradas.csv")

# Crear rango de fechas semanales (52 semanas)
fechas = pd.date_range(start="2024-01-01", periods=52, freq="W")

# Generar dataset sintético
datos_sinteticos = []
for _, row in df.iterrows():
    for fecha in fechas:
        cantidad = max(0, np.random.normal(row["demanda_promedio"], row["desviacion_demanda"]))
        datos_sinteticos.append({
            "fecha": fecha,
            "sku": row["sku"],
            "tienda": row["tienda"],
            "cantidad_vendida": round(cantidad, 2)
        })

ventas_sinteticas = pd.DataFrame(datos_sinteticos)
ventas_sinteticas.to_csv("ventas.csv", index=False)

print("✅ Archivo 'ventas.csv' generado correctamente con", len(ventas_sinteticas), "registros.")
print(ventas_sinteticas.head(10))

✅ Archivo 'ventas.csv' generado correctamente con 52000 registros.
       fecha       sku     tienda  cantidad_vendida
0 2024-01-07  HM000001  TIENDA001              0.80
1 2024-01-14  HM000001  TIENDA001              2.48
2 2024-01-21  HM000001  TIENDA001              0.14
3 2024-01-28  HM000001  TIENDA001              2.48
4 2024-02-04  HM000001  TIENDA001              2.20
5 2024-02-11  HM000001  TIENDA001              0.43
6 2024-02-18  HM000001  TIENDA001              1.02
7 2024-02-25  HM000001  TIENDA001              2.10
8 2024-03-03  HM000001  TIENDA001              1.80
9 2024-03-10  HM000001  TIENDA001              2.05


## Forecasting ARIMA (resultados_forecast.csv)

In [ ]:
# ===============================================================
# Pronóstico de demanda con ARIMA
# ===============================================================
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

# Leer las ventas sintéticas
df = pd.read_csv("ventas.csv")
df["fecha"] = pd.to_datetime(df["fecha"])

resultados = []

# Generar forecast por SKU-Tienda
for (sku, tienda), grupo in df.groupby(["sku", "tienda"]):
    grupo = grupo.sort_values("fecha")
    y = grupo["cantidad_vendida"].values

    # Validar longitud mínima
    if len(y) < 10 or y.sum() == 0:
        print(f"⏭ Sin datos suficientes para {sku} - {tienda}")
        continue

    try:
        modelo = ARIMA(y, order=(1,1,1))
        modelo_fit = modelo.fit()
        pred = modelo_fit.forecast(steps=8)
        fechas_futuras = pd.date_range(start=grupo["fecha"].iloc[-1], periods=8, freq="W")

        for i, f in enumerate(fechas_futuras):
            resultados.append({
                "sku": sku,
                "tienda": tienda,
                "fecha_predicha": f,
                "semana_a_futuro": i+1,
                "prediccion_ARIMA": round(float(pred[i]), 2)
            })
        print(f" Predicción generada para {sku} - {tienda}")
    except Exception as e:
        print(f" Error en {sku}-{tienda}: {e}")

# Guardar resultados
df_forecast = pd.DataFrame(resultados)
df_forecast.to_csv("resultados_forecast.csv", index=False)
print("\n Archivo 'resultados_forecast.csv' generado correctamente.")
print("Filas:", len(df_forecast))
print(df_forecast.head(10))


✅ Predicción generada para HM000001 - TIENDA001
✅ Predicción generada para HM000001 - TIENDA002
✅ Predicción generada para HM000001 - TIENDA002
✅ Predicción generada para HM000001 - TIENDA003
✅ Predicción generada para HM000001 - TIENDA003
✅ Predicción generada para HM000001 - TIENDA004
✅ Predicción generada para HM000001 - TIENDA004
✅ Predicción generada para HM000001 - TIENDA005
✅ Predicción generada para HM000001 - TIENDA005
✅ Predicción generada para HM000001 - TIENDA006
✅ Predicción generada para HM000001 - TIENDA006
✅ Predicción generada para HM000001 - TIENDA007
✅ Predicción generada para HM000001 - TIENDA007
✅ Predicción generada para HM000001 - TIENDA008
✅ Predicción generada para HM000001 - TIENDA008
✅ Predicción generada para HM000001 - TIENDA009
✅ Predicción generada para HM000001 - TIENDA009
✅ Predicción generada para HM000001 - TIENDA010
✅ Predicción generada para HM000001 - TIENDA010
✅ Predicción generada para HM000002 - TIENDA001
✅ Predicción generada para HM000002 - TI

## Validación automática

In [ ]:
# ===============================================================
# 🧩 Validar resultados del forecasting
# ===============================================================
import pandas as pd

try:
    df_forecast = pd.read_csv("resultados_forecast.csv")
    print(f" Total filas: {len(df_forecast)}")
    print(df_forecast.head(10))
except Exception as e:
    print(f" Error al leer el archivo: {e}")

# Diagnóstico adicional (ventas totales)
df_ventas = pd.read_csv("ventas.csv")
resumen = df_ventas.groupby("sku")["cantidad_vendida"].sum().sort_values(ascending=False).head(10)
print("\n Ventas totales (top 10 SKU):\n", resumen)


📊 Total filas: 8000
        sku     tienda fecha_predicha  semana_a_futuro  prediccion_ARIMA
0  HM000001  TIENDA001     2024-12-29                1              1.78
1  HM000001  TIENDA001     2025-01-05                2              1.75
2  HM000001  TIENDA001     2025-01-12                3              1.75
3  HM000001  TIENDA001     2025-01-19                4              1.75
4  HM000001  TIENDA001     2025-01-26                5              1.75
5  HM000001  TIENDA001     2025-02-02                6              1.75
6  HM000001  TIENDA001     2025-02-09                7              1.75
7  HM000001  TIENDA001     2025-02-16                8              1.75
8  HM000001  TIENDA002     2024-12-29                1              1.84
9  HM000001  TIENDA002     2025-01-05                2              1.68

🔍 Ventas totales (top 10 SKU):
 sku
HM000005    1007.50
HM000081     994.75
HM000039     978.90
HM000067     949.55
HM000014     945.91
HM000035     937.41
HM000034     933.54
